In [1]:
import pandas as pd
import numpy as np
from numpy import random
import gensim
from gensim import corpora
from gensim.models import Word2Vec
import nltk
from nltk.corpus import stopwords
import string
from nltk.tokenize.toktok import ToktokTokenizer

import glob
from joblib import Parallel, delayed
import os
from tqdm import tqdm

import itertools
import multiprocessing
from itertools import islice

cpu_count = multiprocessing.cpu_count()

DATA_FOLDER = '../data/'

In [2]:
# all_docs = glob.iglob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)
# corpus_file = open(DATA_FOLDER + 'corpus.txt', 'w')
# for item in all_docs:
#     corpus_file.write("%s\n" % os.path.relpath(item, DATA_FOLDER))

In [3]:
# from nltk.tokenize.toktok import ToktokTokenizer 
# toktok = ToktokTokenizer()
# toktok.tokenize('устройство по п . 3 , отличать тем , что оно снабжать устанавливать в головка винт , в который размещать шарик и ролик .  . ')

In [4]:
stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list.extend(list(string.punctuation))

punkts = [s for s in string.punctuation if s not in '.!?']


class Corpus(gensim.corpora.TextCorpus):
    def get_texts(self):
        for filename in tqdm(self.input): # for each relevant file
            yield tokenize(open(filename).read())


def tokenize(file_text):
    tokens = nltk.word_tokenize(file_text)

    tokens = [validate_word(word) for word in tokens if check_word(word)]
    tokens = filter(lambda s: s != '', tokens)
    
    return tokens

            
def validate_word(word):
    word = word.replace("«", "").replace("»", "").replace("`", '').replace(".", '')
    if len(word) == 1:
        return ''

    return word


def check_word(word):
    if word in stop_list:
        return False
    if any(char.isdigit() for char in word):
        return False
    
    return True

In [5]:
%%time
# dfolder = os.path.join(DATA_FOLDER, 'FIPS/Inventions patents_txt_output/0c/0c')
# all_docs = glob.glob(dfolder + '/**/*.txt', recursive=True)

all_docs = glob.glob(os.path.join(DATA_FOLDER, 'FIPS') + '/**/*.txt', recursive=True)

CPU times: user 7.82 s, sys: 2.43 s, total: 10.3 s
Wall time: 10.3 s


In [6]:
%%time
corpus = Corpus(all_docs)

 22%|██▏       | 266609/1194423 [4:29:10<10:48:50, 23.83it/s]             

KeyboardInterrupt: 

In [7]:
%%time
corpus.dictionary.save(os.path.join(DATA_FOLDER, 'dict.dict'))

NameError: name 'corpus' is not defined

In [8]:
%%time
corpora.MmCorpus.serialize(os.path.join(DATA_FOLDER, 'corpus.mm'), corpus)

NameError: name 'corpus' is not defined

In [9]:
corpus = corpora.MmCorpus(os.path.join(DATA_FOLDER, 'corpus.mm'))
print(corpus)

FileNotFoundError: [Errno 2] No such file or directory: '../data/corpus.mm'

 22%|██▏       | 266609/1194423 [4:29:26<15:37:39, 16.49it/s]